# Анализ текстов 
1. Загрузите объекты из новостного датасета 20 newsgroups, относящиеся к категориям "космос"и "атеизм"(инструкция приведена выше).
2. Вычислите TF-IDF-признаки для всех текстов. Обратите внимание, что в этом задании мы предлагаем вам вычислить TF-IDF по всем данным. При таком подходе получается, что признаки на обучающем множестве используют информацию из тестовой выборки — но такая ситуация вполне законна, поскольку мы не используем значения целевой переменной из теста. На практике нередко встречаются ситуации, когда признаки объектов тестовой выборки известны на момент обучения, и поэтому можно ими пользоваться при обучении алгоритма.
3. Подберите минимальный лучший параметр C из множества [10^−5, 10^−4, ...10^4, 10^5] для SVM с линейным ядром (kernel=’linear’) при помощи кросс-валидации по 5 блокам. Укажите параметр random_state=241 и для SVM, и для KFold. В качестве меры качества используйте долю верных ответов (accuracy).
4. Обучите SVM по всей выборке с лучшим параметром C, найденным на предыдущем шаге.
5. Найдите 10 слов с наибольшим по модулю весом. Они являются ответом на это задание. Укажите их через запятую, в нижнем регистре, в лексикографическом порядке.


In [47]:
from sklearn import datasets
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
import numpy as np

In [48]:
newsgroups = datasets.fetch_20newsgroups(subset='all', categories=['alt.atheism', 'sci.space'])

X = newsgroups.data    # массив с текстами
y = newsgroups.target  # номер класса

# ...   exchange every word from data with a number (TF-IDF)   ...
# ...   TF-IDF = (term frequency) * (inverse document frequency) ...

vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform(X)
X

<1786x28382 sparse matrix of type '<class 'numpy.float64'>'
	with 303138 stored elements in Compressed Sparse Row format>

In [50]:
# ...   подбор параметров   ...
parameters_grid = {'C': np.power(10.0, np.arange(-5, 6))}

cv = KFold(n_splits=5, shuffle=True, random_state=241)
clf = SVC(kernel='linear', random_state=241)
gs = GridSearchCV(clf, parameters_grid, scoring='accuracy', cv=cv)
gs.fit(X, y)

GridSearchCV(cv=KFold(n_splits=5, random_state=241, shuffle=True),
             estimator=SVC(kernel='linear', random_state=241),
             param_grid={'C': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05])},
             scoring='accuracy')

In [83]:
gs.best_params_['C']

1.0

In [95]:
# ...   fitting   ...
clf = SVC(kernel='linear', random_state=241, C=gs.best_params_['C'])
clf.fit(X,y)

# clf.coef_ - разреженная и неотсортированная матрица
coefs = abs(clf.coef_.todense().A1)   # A1 - поле со значениями весов
coefs = np.argsort(coefs)[:-11:-1]
coefs

array([24019, 12871,  5088,  5093, 17802, 23673, 21850,  5776, 15606,
       22936])

In [106]:
feature_mapping = vectorizer.get_feature_names()
main_words = []
for item in coefs:
    main_words.append(feature_mapping[item])

main_words.sort()
main_words

['atheism',
 'atheists',
 'bible',
 'god',
 'keith',
 'moon',
 'religion',
 'sci',
 'sky',
 'space']